In [8]:
import pandas as pd
#a=pd.read_csv('abc_news.csv')
a=pd.read_csv('abc_news_contents_1.csv')
#a=pd.read_csv('CNN_news_with_content.csv')

data=a.iloc[:,3]
print(data.shape)

(385,)


In [4]:
import spacy

# spaCy 언어 모델 로드 (en_core_web_sm 사용 예시)
nlp = spacy.load("en_core_web_sm")

# 텍스트 예제
text = data[0]

# 특정 단어 제거 함수
def remove_words(doc, words_to_remove):
    # words_to_remove는 삭제하려는 단어 리스트
    return [token.text for token in doc if token.text not in words_to_remove]

# 토큰화 및 단어 제거
words_to_remove = ['who','those','they','If','He','I','’','’ve','also','—','we','For','has','We','said','e','as','In','which','’re','On','By','can',':','n’t','this','"','•','Also','||','so','than','Those','was','via','do','not','It','AI','she','+','her','And','how','my','had','now','But','\n','--','n\'t','Albertsons','Kroger','”','”','“','“','?','\'t','(',')','but','\'re','$','%','&','!','your','you','it','his','\'s','he','_',' ',"by", "the","'",',','an','or','and','its','from','a','to','in','for','.','The','that','’s','at','of','be','about','new','on',';','with','are','more','is','-']  # 제거하고자 하는 단어 리스트

tokens=[]
for i in range(len(data)):
    doc=nlp(data[i])
    filtered_tokens = remove_words(doc, words_to_remove)
    tokens.append(filtered_tokens)

# 결과 출력
print(tokens)

[['Charred', 'crisp', 'hundreds', 'brittle', 'ancient', 'scrolls', 'would', 'crumble', 'if', 'one', 'were', 'attempt', 'unfurl', 'them', 'any', 'trace', 'script', 'would', 'nearly', 'illegible', 'Herculaneum', 'scrolls', 'known', 'still', 'remain', 'unopened', 'thanks', 'powerful', 'tool', 'artificial', 'intelligence', 'their', 'contents', 'lie', 'within', 'reach', 'Using', 'high', 'resolution', 'X', 'rays', 'trio', 'researchers', 'decoded', '2023', '2,000', 'characters', 'rolled', 'scrolls', 'remarkable', 'feat', 'laid', 'bare', 'first', 'full', 'passages', 'papyri', 'survived', 'eruption', 'Mount', 'Vesuvius', 'AD', '79', 'artifacts', 'recovered', 'building', 'believed', 'have', 'been', 'house', 'Julius', 'Caesar', 'father', 'law', 'form', 'unprecedented', 'cache', 'information', 'ancient', 'Rome', 'Greece', 'Computer', 'scientists', 'launched', 'Vesuvius', 'Challenge', 'competition', 'designed', 'accelerate', 'deciphering', 'process', 'hope', '90', 'four', 'scrolls', 'will', 'unlock

In [6]:
#!pip install gensim
from gensim import corpora

dictionary = corpora.Dictionary(tokens) # 토큰 단어와 gensim 내부 아이디 매칭
dictionary.filter_extremes(no_below=2, no_above=0.1) # 빈도 2이상 포함, 전체 50% 이상 단어 제거
corpus = [dictionary.doc2bow(token) for token in tokens]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 3), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 3), (36, 1), (37, 1), (38, 1), (39, 3), (40, 1), (41, 2), (42, 1), (43, 2), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1), (85, 1), (86, 2), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 3), (96, 1), (97, 1), (98, 3), (99, 1), (100, 1), (101, 2), (102, 4), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 2), (109, 1), (110, 4)

In [ ]:
# 응집도 코드
import gensim
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt


for num_topics in range(3, 15):
    lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary)
    
    # 4. Coherence Score 계산
    coherence_model = CoherenceModel(model=lda_model, texts=tokens, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    
    print(f"토픽 수: {num_topics}, Coherence Score: {coherence_score}")


In [2]:
import gensim
final_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10)
final_model.print_topics()

#%pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

prepared_data = gensimvis.prepare(final_model, corpus, dictionary)
pyLDAvis.display(prepared_data)

NameError: name 'corpus' is not defined